<h1><center>Multi-Period Demand Response Management</center></h1>


# 1. Stackelberg Game 
Let $\scr{K}=\{1,2,\dots,K\}$ be the set of companies, $\scr{N}=\{1,2,\dots,N\}$ be the set of consumers, and $\scr{T}=\{1,2,\dots,T\}$ be the finite set of time slots.

We formulate a static Stackelberg game between companies (the leaders) and their consumers (the followers) to find revenue maximizing prices and optimal demands. In Stackelberg games, the leader(s) first announce their decisions to the follower(s), and then the followers respond. In our game, the leaders send price signals to the consumers, who respond optimally by choosing their demands. To capture the  market competition between the companies, we let them play a price-selection Nash game. The equilibrium point of the price-selection game is what companies announce to their consumers.




## 1.1 Consumer-Side 

Each energy consumer $n\in \scr{N}$ receives all price signals from each company $k\in \scr{K}$ at each time slot $t\in \scr{T}$ and aims to select his corresponding utility-maximizing demand $d_{n,k}(t)\geq0$ for each time slot from each company, subject to budget and energy need constraints. Denote company $k$'s price at time $t$ by $p_k(t)$. Let $B_n \geq 0$  and $E^{{\rm min}}_n \geq 0$ denote, respectively, consumer $n$'s budget and minimum energy need for the entire time-horizon. The utility of consumer $n$ is defined as 
$$ U_{{\rm consumer},n}=\gamma_n\sum_{k\in \scr{K}}\sum_{t\in \scr{T}}\ln(\zeta_n+d_{n,k}(t))$$
where $\gamma_n$ and $\zeta_n\geq1$ are consumer specific parameters. Consumer $n$ aims to achieve the highest payoff while meeting the threshold of minimum amount of energy and not exceeding a certain budget. To be more precise,
given $B_n \geq 0$, $E^{{\rm min}}_n \geq 0$, and $p_k(t)>0$, the consumer-side optimization problem is formulated as follows:

\begin{eqnarray}&{\text{maximize}}   \ \ \ \ \ & U_{{\rm consumer},n}, \\
 &\text{subject to} &   \ \sum_{k\in \scr{K}}\sum_{t\in \scr{T}}p_k(t)d_{n,k}(t)\leq B_n \\
&&\sum_{k\in \scr{K}}\sum_{t\in \scr{T}}d_{n,k}(t)\geq \,E^{{\rm min}}_n, \\
&& d_{n,k}(t)\geq 0,\;\;   \\
&& \text{for each } \ t \in \scr{T}, \ k \in \scr{K} \end{eqnarray}

For the remaining of this notebook, we let $\zeta_n=\gamma_n=1$ for each $n$.

## 1.2 Company-Side 

Letting ${\bf{p_{-k}}}$ denote the prices set by other companies, the total revenue for company $k$ is given by

$$U_{{\rm company},k}=\sum_{t\in \scr{T}}p_k(t)\sum_{n\in \scr{N}}d_{n,k}(p_k,{\bf{p_{-k}}},t).$$

Given the power availability of UC $k$ at period $t$, denoted by $G_k(t)$, and for a fixed ${\bf{p_{-k}}}$, company $k$'s problem is


\begin{eqnarray}&{\text{maximize}}   \ \ \ \ \ & U_{{\rm company},k} (p_k,{\bf{p_{-k}}}), \\
&\text{subject to} &   \ \sum_{n\in \scr{N}} d_{n,k}(p_k,{\bf{p_{-k}}},t) \leq G_k(t) \\
&& p_k(t)> 0, \\
&& \text{for each } \ t \in \scr{T}. \end{eqnarray}

  The goal of each company is to maximize its revenue and hence maximize its profit. Additionally, because of the market competition, the prices announced by other companies also affect the determination of the price at company $k$. Thus, company $k$'s price selection is actually a response to what other competitors in the market have announced; this response is also constrained by the availability of power. Thus, what we have is a Nash game among the utility companies.

<b> In this notebook, we numerically implement the closed-form solutions and algorithm discussed in [Alshehri et. al.] for various scenarios.

In [1]:
#Calling the necessary libraries 

import numpy as np 
import scipy.io
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
%matplotlib inline


<b>Now, we define the optimal demand functions $d^*_{n,k}(t)$ and prices $p^*_{k}(t)$, and other necessary functions. <b>

In [2]:
# In this cell, we only define the functions 

# Denote the number of consumers by N, the number of utility companies by K, and the number of periods by T. 
# We take \zeta_n's and \gamma_n's to be 1 for all consumers 

def d(n,k,t):
    
    y=(B[n-1]+np.sum(P))/(K*T*P[k-1,t-1])-1    
    
    return y

def p(k,t):
    
    y=(np.sum(B)/(G[k-1,t-1]+N))*(1/(K*T-np.sum(N/(G+N))))   
    
    return y

def Un(n):
    
    demand=np.zeros((K,T))
    for i in range(T):
        for j in range(K):
            demand[j,i]=d(n,j,i)
    
    y=np.sum(np.log(1+demand))
    
    return y

def Uk(k):
    
    demand=np.zeros((N,T))
    for i in range(T):
        for j in range(N):
            demand[j,i]=d(j,k,i)

    demand_t=np.zeros(T)
    for i in range(T):
        demand_t[i]=np.sum(demand[:,i])

    y=np.dot(P[k-1,:],demand_t)

    return y

## 2. Case Study: EcoGrid EU

In [3]:
T,N,K=24,2000,1
B,G,P,P_Eco,Payment_Eco,Payment_Stackelberg=np.zeros(N),np.zeros((K,T)),np.zeros((K,T)),np.zeros((K,T)),np.zeros((T)),np.zeros((T))


# Flexible Power from EcoGrid 
G=([1900,1800,1900,2000,2200,2400,2450,2400,2350,2375,2375,2400,2550,2700,2650,2550,2450,2350,2250,2150,2100,2000,1850,1900])
G=np.reshape(G,(K,T))

# Prices from EcoGrid 
P_Eco=([230,280,230,230,250,370,360,370,380,360,350,330,355,370,350,330,270,310,250,230,220,240,300,240])
P_Eco=np.reshape(P_Eco,(K,T))/1000

E_total=np.sum(G)

# Find necessary budgets from Theorem 1
B[0:N]=(((E_total/N+K*T)/np.sum(1/(K*T*P_Eco)))-np.sum(P_Eco))

plt.figure(figsize=(20,5))    

# Visualizing power
ax = plt.subplot(131)

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

   
  
# plt.plot(G[0,:],color='g',label='Power availability',linewidth=3)
plt.plot(G[0,:],color='r',linewidth=3,alpha=0.75)
plt.grid(True)    
plt.ylabel('Total Power (kW)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
# plt.legend(fontsize=15,loc='best')
plt.ylim(np.min(G)-100,np.max(G)+100)
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 



# Optimal Prices 
for t in range(T):
    for k in range(K):
        P[k,t]=p(k+1,t+1)

# Visualizing prices
ax = plt.subplot(132)

plt.ylim(0.15,.45)
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    
  

plt.plot(P[0,:],color='grey',label='Stackelberg game ',linewidth=3)
plt.plot(P_Eco[0,:],color='k',label='EcoGrid EU',linewidth=3,alpha=1)
plt.grid(True)    
plt.ylabel('Price (DKK/kWh)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
plt.legend(fontsize=18,loc='best')



# Visualizing savings 


ax = plt.subplot(133)

k=0
for t in range(T):
        Payment_Stackelberg[t]=p(k+1,t+1)*G[k,t]+Payment_Stackelberg[t-1]
        Payment_Eco[t]=P_Eco[k,t]*G[k,t]+Payment_Eco[t-1]
  
i=np.zeros([T])
for j in range(T):
    i[j]=j
        
plt.plot(Payment_Stackelberg[:],color='grey',label='Stackelberg game',linewidth=3)
plt.plot(Payment_Eco[:],color='k',label='EcoGrid EU',linewidth=3)


ax.fill_between(i,Payment_Eco,Payment_Stackelberg, where=Payment_Eco >= Payment_Stackelberg, alpha=0.5, color='green')
ax.fill_between(i,Payment_Eco,Payment_Stackelberg, alpha=0.5,where=Payment_Eco <= Payment_Stackelberg, color='red')

plt.ylabel('Cumulative Consumers Payments (DKK)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 
plt.legend(fontsize=18,loc='best')
plt.ticklabel_format(style='sci', fontsize=30, axis='y', scilimits=(0,0))
ax.yaxis.offsetText.set_fontsize(18)

plt.grid(True)    

np.sum(Payment_Eco)/np.sum(Payment_Stackelberg)
plt.savefig('EcoGridEU_Price_Power.pdf')


TypeError: ticklabel_format() got an unexpected keyword argument 'fontsize'

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 1440x360 with 3 Axes>

## 2.1 Influence of the Number of Periods

In [ ]:

# # # We study the influence of the number of periods at the NW of 
# N=2000
# K=4
# B=np.zeros(N)
# Max_T=50
# Utility=np.zeros((Max_T+1,N))
# Revenue=np.zeros((Max_T+1,K))
# Prices_NE=np.zeros((Max_T+1,K))
# Power_NE=np.zeros((Max_T+1,K))
# E_total=np.sum(([1900,1800,1900,2000,2200,2400,2450,2400,2350,2375,2375,2400,2550,2700,2650,2550,2450,2350,2250,2150,2100,2000,1850,1900]))

# for i in range(Max_T+1):
#     if i==0:
#         continue
#     T=i
#     G=np.zeros((K,T))
#     P=np.zeros((K,T))
#     B[0:400],B[400:800],B[800:1200],B[1200:1600],B[1600:2000]=4,5,6,7,8
#     G[0,:],G[1,:],G[2,:],G[3,:]=(36/59)*E_total/T,(16/59)*E_total/T,(5/59)*E_total/T,(2/59)*E_total/T
#     for t in range(T):
#         for k in range(K):
#             P[k,t]=p(k+1,t+1)
    
#     for k in range(K):
#         Revenue[i,k]=Uk(k+1)
#         Prices_NE[i,k]=p(k+1,1)
#         Power_NE[i,k]=G[k,0]
    
#     for n in range(N):
#         Utility[i,n]=Un(n+1)
 

# Visualizing prices and power at NE 

plt.figure(figsize=(20,10))    

ax = plt.subplot(221)

plt.plot(Prices_NE[:,0],color='C%s'%(1),label='$k=1$',linewidth=3)
plt.plot(Prices_NE[:,1],color='C%s'%(2),label='$k=2$',linewidth=3)
plt.plot(Prices_NE[:,2],color='C%s'%(3),label='$k=3$',linewidth=3)
plt.plot(Prices_NE[:,3],color='C%s'%(4),label='$k=4$',linewidth=3)

plt.ylim(np.min(Prices_NE)-0.1, np.max(Prices_NE)+0.1)
plt.xlim((1,Max_T))
plt.xticks(np.arange(1, 51, step=3))  
  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    
  
        
plt.grid(True)    
plt.ylabel('Price per Period (DKK/kWh)',fontsize=18)
plt.xlabel('Number of Periods ($T$)',fontsize=18)
plt.legend(fontsize=18,loc='best')


# Visualizing revenues

  
# Remove the plot frame lines.  
ax = plt.subplot(222)    
  
plt.ylim(np.min(Revenue)-100, np.max(Revenue)+1)
plt.xlim(1,Max_T,1)
 
  
# # Make sure your axis ticks are large enough to be easily read.    
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    
  

        
plt.plot(Revenue[:,0],color='C%s'%(1),label='$k=1$',linewidth=3)
plt.plot(Revenue[:,1],color='C%s'%(2),label='$k=2$',linewidth=3)
plt.plot(Revenue[:,2],color='C%s'%(3),label='$k=3$',linewidth=3)
plt.plot(Revenue[:,3],color='C%s'%(4),label='$k=4$',linewidth=3)
plt.grid(True)    
plt.ylabel('Revenue (DKK)',fontsize=18)
plt.xlabel('Number of Periods (T)',fontsize=18)
plt.legend(fontsize=18,loc='best')
plt.ticklabel_format(style='sci', fontsize=30, axis='y', scilimits=(0,0))
plt.xticks(np.arange(1, 51, step=3))  
ax.yaxis.offsetText.set_fontsize(18)


# Power per priod at NE 
ax = plt.subplot(223)

plt.plot(Power_NE[:,0],color='C%s'%(1),label='$k=1$',linewidth=3)
plt.plot(Power_NE[:,1],color='C%s'%(2),label='$k=2$',linewidth=3)
plt.plot(Power_NE[:,2],color='C%s'%(3),label='$k=3$',linewidth=3)
plt.plot(Power_NE[:,3],color='C%s'%(4),label='$k=4$',linewidth=3)

plt.ylim(np.min(Power_NE)-100, np.max(Power_NE)+1)
plt.xlim(1,Max_T,1)
   
plt.ticklabel_format(style='sci', fontsize=30, axis='y', scilimits=(0,0))
plt.xticks(np.arange(1, 51, step=3))  

plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    
# ax.ticklabel_format(style='sci',scilimits=(0,0))

        
plt.grid(True)    
plt.ylabel('Power per Period (kW)',fontsize=18)
plt.xlabel('Number of Periods ($T$)',fontsize=18)
plt.legend(fontsize=18,loc='best')
ax.yaxis.offsetText.set_fontsize(18)

# Visualizing utilities

ax = plt.subplot(224)
  
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.ylim(4, np.max(Utility)+1)
plt.xticks(np.arange(1, 51, step=3))  
  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    


        
plt.plot(Utility[:,0],color='C%s'%(6),label='$n=1-400$',linewidth=3)
plt.plot(Utility[:,401],color='C%s'%(7),label='$n=401-800$',linewidth=3)
plt.plot(Utility[:,801],color='C%s'%(8),label='$n=801-1200$',linewidth=3)
plt.plot(Utility[:,1201],color='C%s'%(4),label='$n=1201-1600$',linewidth=3)
plt.plot(Utility[:,1601],color='C%s'%(5),label='$n=1601-2000$',linewidth=3)

plt.grid(True)  

plt.ylabel('Consumer Utility',fontsize=18)
plt.xlabel('Number of Periods ($T$)',fontsize=18)
plt.legend(fontsize=18,loc='upper left')


plt.savefig('Influence_of_T.pdf')


### 2.2 Distributed Algorithm

For each iteration $i\in\{0,1,\ldots\}$, denote the demand from user $n$ at time $t$ from company $k$ by $d^{(i)}_{n,k}(t)$, and the price announced by company $k$ and time $t$ by $p^{(i)}_k(t)$. In our algorithm,  $p^{(1)}_k(t)$ is chosen arbitrarily for each company $k\in \scr{K}$ and time $t\in\scr{T}$. Based on the initial price selection, $d^{(1)}_{n,k}$'s are computed. Then, the prices are sequentially updated using the following update rule: 
$$p^{(i+1)}_k(t)=p^{(i)}_k(t)+\frac{\sum_{n\in \scr{N}}d^{(i)}_{n,k}(t)-G_k(t)}{\epsilon^{(i)}_{k,t}},$$
where $\epsilon^{(i)}_{k,t}>0$ is $$ \epsilon^{(i)}_{k,t} =  \frac{G_k(t)+N}{p^{(i)}_k(t)} + \delta,$$
where $\delta\geq0$.


In [ ]:

T,N,K=1,2000,4
B,G=np.zeros(N),np.zeros((K,T))
B[0:400],B[400:800],B[800:1200],B[1200:1600],B[1600:2000]=4,5,6,7,8


G[0,:],G[1,:],G[2,:],G[3,:]=(36/59)*E_total/T,(16/59)*E_total/T,(5/59)*E_total/T,(2/59)*E_total/T


Max_iter=40
P,demand=np.ones([K,T])*5,np.zeros((N))
P_in,demand_in=np.ones([K,T])*5,np.zeros((N))

#Logging prices for each iteration for each company
P_iter_1,P_iter_2,P_iter_3,P_iter_4=np.ones(Max_iter),np.ones(Max_iter),np.ones(Max_iter),np.ones(Max_iter)

# computing the prices with delta=100. 

delta=1000
for i in range(Max_iter):
    if i==0:
        P_iter_1[i]=P_in[0,0]
        P_iter_2[i]=P_in[1,0]
        P_iter_3[i]=P_in[2,0]
        P_iter_4[i]=P_in[3,0]
    else:    
        for t in range(T):
            for k in range(K):
                for n in range(N):
                    demand[n]=d(n+1,k+1,t+1)
                epsilon=(G[k,t]+N)/P[k,t]+delta
                P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
        P_iter_1[i]=P[0,0]
        P_iter_2[i]=P[1,0]
        P_iter_3[i]=P[2,0]
        P_iter_4[i]=P[3,0]

    
# Visualizing the outcomes


plt.figure(figsize=(15, 5))    

ax = plt.subplot(121)

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

# plt.ylim(np.min([P_iter_1,P_iter_2,P_iter_3,P_iter_4])-0.1,np.max([P_iter_1,P_iter_2,P_iter_3,P_iter_4])+0.1)
plt.xlim(0,Max_iter,1)  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    

        
plt.plot(P_iter_1[:],color='C%s'%(1),label='$k=1$',linewidth=3)
plt.plot(P_iter_2[:],color='C%s'%(2),label='$k=2$',linewidth=3)
plt.plot(P_iter_3[:],color='C%s'%(3),label='$k=3$',linewidth=3)
plt.plot(P_iter_4[:],color='C%s'%(4),label='$k=4$',linewidth=3)
plt.grid(True)    
plt.ylabel('Price (DKK/kWh)',fontsize=18)
plt.xlabel('Iteration $i$',fontsize=18)
plt.title('$\delta=1000$ (fast convergence)',fontsize=18)
plt.legend(fontsize=18,loc='best')


# # Increase delta to 10000 
#Logging prices for each iteration for each company
P_iter_1,P_iter_2,P_iter_3,P_iter_4=np.ones(Max_iter),np.ones(Max_iter),np.ones(Max_iter),np.ones(Max_iter)

P,demand=np.ones([K,T])*5,np.zeros((N))
delta=10000
for i in range(Max_iter):
    if i==0:
        P_iter_1[i]=P_in[0,0]
        P_iter_2[i]=P_in[1,0]
        P_iter_3[i]=P_in[2,0]
        P_iter_4[i]=P_in[3,0]
    else:    
        for t in range(T):
            for k in range(K):
                for n in range(N):
                    demand[n]=d(n+1,k+1,t+1)
                epsilon=(G[k,t]+N)/P[k,t]+delta
                P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
        P_iter_1[i]=P[0,0]
        P_iter_2[i]=P[1,0]
        P_iter_3[i]=P[2,0]
        P_iter_4[i]=P[3,0]

    


# Plot the prices 

ax = plt.subplot(122)
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.ylim(np.min([P_iter_1,P_iter_2,P_iter_3,P_iter_4])-0.1,np.max([P_iter_1,P_iter_2,P_iter_3,P_iter_4])+0.1)
plt.xlim(0,Max_iter,1)  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    
  

        
plt.plot(P_iter_1[:],color='C%s'%(1),label='$k=1$',linewidth=3)
plt.plot(P_iter_2[:],color='C%s'%(2),label='$k=2$',linewidth=3)
plt.plot(P_iter_3[:],color='C%s'%(3),label='$k=3$',linewidth=3)
plt.plot(P_iter_4[:],color='C%s'%(4),label='$k=4$',linewidth=3)

plt.grid(True)    
plt.ylabel('Price (DKK/kWh)',fontsize=18)
plt.xlabel('Iteration $i$',fontsize=18)
plt.legend(fontsize=18,loc='best')
plt.title('$\delta=10000$ (slower convergence)',fontsize=18)


# # Divergence 

# P,demand=np.random.rand(K,T)*5,np.zeros((N))
# delta=-10000
# for i in range(Max_iter):
#     for t in range(T):
#         for k in range(K):
#             for n in range(N):
#                 demand[n]=d(n+1,k+1,t+1)
#             epsilon=(G[k,t]+N)/P[k,t]+delta
#             P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
#     P_iter_1[i]=P[0,0]
#     P_iter_2[i]=P[1,0]
#     P_iter_3[i]=P[2,0]
#     P_iter_4[i]=P[3,0]


# ax = plt.subplot(133)

  
# ax.get_xaxis().tick_bottom()    
# ax.get_yaxis().tick_left()    

# plt.ylim(np.min([P_iter_1,P_iter_2,P_iter_3,P_iter_4])-0.1,np.max([P_iter_1,P_iter_2,P_iter_3,P_iter_4])+0.1)
# plt.xlim(0,Max_iter,1)  
# plt.xticks(fontsize=18)    
# plt.yticks(fontsize=18)    
  

        
# plt.plot(P_iter_1[:],color='C%s'%(1),label='$k=1$',linewidth=3)
# plt.plot(P_iter_2[:],color='C%s'%(2),label='$k=2$',linewidth=3)
# plt.plot(P_iter_3[:],color='C%s'%(3),label='$k=3$',linewidth=3)
# plt.plot(P_iter_4[:],color='C%s'%(4),label='$k=4$',linewidth=3)

# plt.grid(True)    
# plt.ylabel('Price ($DKK/MWh$)',fontsize=18)
# plt.xlabel('Iteration $i$',fontsize=18)
# plt.legend(fontsize=18,loc='best')
# plt.title('$\delta=-10000$ (divergence)',fontsize=18)
# plt.ticklabel_format(style='sci', fontsize=30, axis='y', scilimits=(0,0))

plt.savefig('algorithm.pdf')


## 3. Case Study: Dutch Smart Grid Pilot

In [ ]:
T,N,K=24,77,1
B,G,P,P_Eco,Payment_Stackelberg,Payment_Eco=np.zeros(N),np.zeros((K,T)),np.zeros((K,T)),np.zeros((K,T)),np.zeros((T)),np.zeros((T))


# Flexible Power from Dutch pilot [Watts]
G=([300,280,260,240,
    220,210,200,230,
    400,410,415,420,
    430,425,420,415,
    400,420,600,450,
    440,430,380,370])

# convert to kW and reshape  
G=77*np.reshape(G,(K,T))/1000

# Prices from Dutch pilot [EUR/kWh]

P_Eco=([0.162,0.16,0.15,0.145,0.143,
       0.145,0.15,0.157,0.156,
       0.155,0.15,0.145,0.14,
       0.135,0.12,0.13,0.135,
       0.18,0.22,0.28,0.36,
       0.3,0.275,0.215])

P_Eco=np.reshape(P_Eco,(K,T))

E_total=np.sum(G)

# Find necessary budgets from Theorem 1
B[0:N]=(((E_total/N+K*T)/np.sum(1/(K*T*P_Eco)))-np.sum(P_Eco))



# Optimal Prices 
for t in range(T):
    for k in range(K):
        P[k,t]=p(k+1,t+1)

# Visualizing prices
plt.figure(figsize=(20,5))    
ax = plt.subplot(132)

plt.ylim(0,.4)
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18)    


plt.plot(P[0,:],color='grey',label='Stackelberg game',linewidth=3)
plt.plot(P_Eco[0,:],color='k',label='Dutch Pilot',linewidth=3,alpha=1)
plt.grid(True)    
plt.ylabel('Price (EUR/kWh)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
plt.legend(fontsize=18,loc='best')



# Visualizing demand
ax = plt.subplot(131)

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

   
dd=np.zeros(T)
for i in range(T):
    dd[i]=d(1,1,i)

    
# plt.plot(G[0,:],color='g',label='Power availability',linewidth=3)
plt.plot(dd,color='r',linewidth=3,alpha=0.75)
plt.grid(True)    
plt.ylabel('Consumer Demand (kW)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
plt.title('Average Consumer',fontsize=18)
# plt.legend(fontsize=18,loc='best')
plt.ylim(np.min(dd)-0.1,np.max(dd)+.10)
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 

ax = plt.subplot(133)

ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

   
for t in range(T):
    Payment_Stackelberg[t]=Payment_Stackelberg[t-1]+dd[t]*P[0,t]
    Payment_Eco[t]=Payment_Eco[t-1]+dd[t]*P_Eco[0,t]

plt.grid(True)    

plt.ylabel('Cumulative Payment (EUR)',fontsize=18)
plt.xlabel('Time of day (Hour)',fontsize=18)
plt.plot(Payment_Stackelberg[:],color='grey',label='Stackelberg game',linewidth=3)
plt.plot(Payment_Eco[:],color='k',label='Dutch Pilot',linewidth=3)

i=np.zeros([T])
for j in range(T):
    i[j]=j

ax.fill_between(i,Payment_Eco,Payment_Stackelberg,alpha=0.5, color='green')

plt.title('Average Consumer',fontsize=18)
plt.legend(fontsize=18,loc='best')
plt.xticks(np.arange(0, T, 3))
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 





plt.savefig('Netherlands_Price_Power.pdf')

In [ ]:
print(E_total/N,B[0])

### 3.1 Distributed Algorithm

In [ ]:

#Convergence of the Algorithm 
Max_iter=50

plt.figure(figsize=(15, 5))    

ax = plt.subplot(121)


#Logging prices for each iteration for each period
P_iter=np.zeros([T,Max_iter])
P,demand=np.ones([K,T])/2,np.zeros((N))



delta,k=100,0
for i in range(Max_iter):
    if i==0:
        P_iter[:,i]=P
    else:    
        for t in range(T):
            for k in range(K):
                for n in range(N):
                    demand[n]=d(n+1,k+1,t+1)
                epsilon=(G[k,t]+N)/P[k,t]+delta
                P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
        P_iter[:,i]=P
    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.ylim(np.min([P_iter])-0.1,np.max([P_iter])+0.1)
plt.xlim(0,40,1)  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 


for i in range(T):
    plt.plot(P_iter[i,:],linewidth=3)
        
plt.ylabel('Price (EUR/kWh)',fontsize=18)
plt.xlabel('Iteration $i$',fontsize=18)
plt.title('$\delta=100$ (fast convergence)',fontsize=18)

plt.grid(True)    


ax = plt.subplot(122)


#Logging prices for each iteration for each period
P_iter=np.zeros([T,Max_iter])


P,demand=np.ones([K,T])/2,np.zeros((N))

delta,k=1000,0
for i in range(Max_iter):
    if i==0:
        P_iter[:,i]=P
    else:    
        for t in range(T):
            for k in range(K):
                for n in range(N):
                    demand[n]=d(n+1,k+1,t+1)
                epsilon=(G[k,t]+N)/P[k,t]+delta
                P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
        P_iter[:,i]=P
    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()    

plt.ylim(np.min([P_iter])-0.1,np.max([P_iter])+0.1)
plt.xlim(0,40,1)  
plt.xticks(fontsize=18)    
plt.yticks(fontsize=18) 


for i in range(T):
    plt.plot(P_iter[i,:],linewidth=3)
        
plt.ylabel('Price (EUR/kWh)',fontsize=18)
plt.xlabel('Iteration $i$',fontsize=18)
plt.title('$\delta=1000$ (slower convergence)',fontsize=18)

plt.grid(True)    


# ax = plt.subplot(133)


# #Logging prices for each iteration for each period
# P_iter=np.zeros([T,Max_iter])


# P,demand=np.random.rand(K,T),np.zeros((N))
# delta,k=-1000,0
# for i in range(Max_iter):
#     for t in range(T):
#         for n in range(N):
#             demand[n]=d(n+1,k+1,t+1)
#         epsilon=(G[k,t]+N)/P[k,t]+delta
#         P[k,t]=P[k,t]+(np.sum(demand)-G[k,t])/epsilon
#         P_iter[t,i]=P[k,t]
    
# ax.get_xaxis().tick_bottom()    
# ax.get_yaxis().tick_left()    

# plt.ylim(np.min([P_iter])-0.1,np.max([P_iter])+0.1)
# plt.xlim(0,40,1)  
# plt.xticks(fontsize=18)    
# plt.yticks(fontsize=18) 


# for i in range(T):
#     plt.plot(P_iter[i,:],linewidth=3)
        
# plt.ylabel('Price (EUR/kWh)',fontsize=18)
# plt.xlabel('Iteration $i$',fontsize=18)
# plt.title('$\delta=-1000$ (divergence)',fontsize=18)
# plt.xticks(fontsize=18)    
# plt.yticks(fontsize=18) 


plt.grid(True)    

plt.savefig('algorithm2.pdf')


##### References 
K. Alshehri, J. Liu, X. Chen, and T. Başar. A Stackelberg game for multi-period demand response management in the smart grid. Proc. 54th IEEE Conference on Decision and Control (CDC'15, Dec 15-18, 2015; Osaka, Japan), pp. 5889-5894.


K. Alshehri, J. Liu, X. Chen, and T. Başar. A Game-Theoretic Method for Multi-Period Demand Response: Revenue Maximization, Power Allocation, and Asymptotic Behavior. https://arxiv.org/abs/1710.00145

